In [26]:
# Use full Excel sheet
import pandas as pd
import string
import re
Cat = pd.read_excel(r'../De-Identified Mohs Full Data.xlsx',sheet_name='Identified Data',header=0) # Only Size at Greatest Dimension column
sample_size = len(Cat)

#txt_string = r'at_GPT_data/at_column_data.txt' # No prompt change
#txt_string = r'at_GPT_data/at_column_data_change_4.txt' # Prompt change 3

#txt_string = r'GPT_data/location_data(1).txt'
#txt_string = r'GPT_data/aggressive_tumor_data(1).txt'
#txt_string = r'GPT_data/dfsp_data(1).txt'
#txt_string = r'GPT_data/recurrent_tumor_data(1).txt'

#txt_string = r'GPT_data/cf_data(1).txt'
#txt_string = r'GPT_data/ae_data(3).txt'
#txt_string = r'GPT_data/ms_data(1).txt'
#txt_string = r'GPT_data/ent_data(1).txt'

txt_string = r'GPT_data/tumor_size_data(1).txt'

#ground_truth_string = 'Location - Nose, Eyelid, Nail, Lip, Genitalia, or Acral (L)'
#ground_truth_string = 'Aggressive Tumor Pathology Type (AT)'
#ground_truth_string = 'DFSP (D)'
#ground_truth_string = 'Recurrent Tumor (R)'

#ground_truth_string = 'Additional C&F'
#ground_truth_string = 'Additional Excision'
#ground_truth_string = 'Multiple Sites'
#ground_truth_string = 'Coordinated repair with ENT/oculoplastics (C)'

ground_truth_string = 'Size at Greatest Dimension (S) ***Be  sure this is from the note and not the Mohs episode!'

only_A_flag = False # Only check Addendum if True
# Array of indexes containing patients with notes that have problems
skip_patients_array = [7,8,22,23,32,34,38,39,49,50,52,55,62,66,73,80,88,89,101,102,107,110,121,123,131,133,135,
                       136,137,140,150,155,157,164,170,184,187,193,200,206,210,214,227,228,229,234,252,261,265,279,287,292,293]
skip_patients = True # Set to True to skip bad patient notes, False to keep them in

In [27]:
# For testing which voting method works best
# Clean output
def cleanResponse(line):
    clean_output = [word.lower().strip(string.punctuation) for word in line.split() if word.lower().strip(string.punctuation) in ['yes', 'no']]
    if "yes" in clean_output:
        return "y"
    elif "no" in clean_output:
        return "n"
    else:
        return "error"

def checkLine(string, line, array): # Cleans response an adds to appropriate array
    if string in line:
        clean_output = cleanResponse(line)
        array.append(clean_output)
        
def checkLineSize(string, line, array): # Cleans response an adds to appropriate array (only for Size question)
    if string in line:
        clean_output = line.split(":", 1)[1].strip() # Extract the part after the patint note column label
        array.append(clean_output)

def getIndex(string, line):
    if string in line:
        index = re.findall(r'\d+', line)
        return int(index[0])
    return None

with open(txt_string, "r") as log_file:
    file_content = log_file.readlines() # Read all lines, turn them into strings

PE_unfiltered_array = []
AP_unfiltered_array = []
A_unfiltered_array = []
true_results = [0 for i in range(sample_size)]
index = 0

# For all other questions
def getData():
    for line in file_content:
        checkLine("PE Unfiltered", line, PE_unfiltered_array) # Fill in arrays with GPT responses from txt file
        checkLine("A&P Unfiltered", line, AP_unfiltered_array)
        checkLine("A Unfiltered", line, A_unfiltered_array)
        index = getIndex("Index", line)

        if index is not None:
            true_results[index] = Cat.at[index, ground_truth_string] # Get the ground truth for current index
            if pd.isna(true_results[index]):
                true_results[index] = "NaN error"
                #print(f"NaN at index {index}")

            elif isinstance(true_results[index], str):
                raw = true_results[index]
                parts = [p.lower().strip() for p in raw.split('/')]
                if any('y' in p for p in parts):  # This matches "y", "yes", "y/n", etc.
                    true_results[index] = "y"
                else:
                    true_results[index] = "n"

# Only for Size question
def getDataSize():
    for line in file_content:
        checkLineSize("PE Unfiltered", line, PE_unfiltered_array) # Fill in arrays with GPT responses from txt file
        checkLineSize("A&P Unfiltered", line, AP_unfiltered_array)
        checkLineSize("A Unfiltered", line, A_unfiltered_array)
        index = getIndex("Index", line)

        if index is not None:
            true_results[index] = Cat.at[index, ground_truth_string] # Get the ground truth for current index
            if pd.isna(true_results[index]):
                true_results[index] = -1 # No valid tumor size provided in Ground Truth
                #print(f"NaN at index {index}")

            if isinstance(true_results[index], (str, float)):
                result_str = str(true_results[index]).strip() # Convert to string to strip whitespaces and process
                
                if result_str:
                    sizes_real_parts = re.split(r'[;/]', result_str)
                    valid_sizes = []

                    for part in sizes_real_parts:
                        part = part.strip()
                        if re.search(r'NA|nan', part, re.IGNORECASE):
                            continue
                        if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', part):
                            continue
                        num_match = re.search(r'(\d*\.?\d+)', part)
                        if num_match:
                            valid_sizes.append(float(num_match.group(1)))

                    largest_size_real = max(valid_sizes) if valid_sizes else -1

                else:
                    largest_size_real = -1 # No valid tumor size provided in Ground Truth
            else:
                largest_size_real = -1 # No valid tumor size provided in Ground Truth
            
            true_results[index] = largest_size_real  # Store largest size in true_results

#getData()
getDataSize()

num_y = true_results.count("y")
num_n = true_results.count("n")
num_nan = true_results.count(-1)
print(f"# of 'Y' in Ground Truth: {num_y}")
print(f"# of 'N' in Ground Truth: {num_n}")
print(f"# of NaN values in Ground Truth: {num_nan}")

print(f"Ground Truth:\n{true_results}")
print(f"PE Unfiltered Array:\n{PE_unfiltered_array}")
print(f"AP Unfiltered Array:\n{AP_unfiltered_array}")
print(f"A Unfiltered Array:\n{A_unfiltered_array}")
# 24 113 281 NaN
# 36 62 67 75 115 126 131 186 189 197 199 207 222 228 239 241 242 243 254 279 286 287 288 Y/N

# of 'Y' in Ground Truth: 0
# of 'N' in Ground Truth: 0
# of NaN values in Ground Truth: 93
Ground Truth:
[0.8, 0.6, 0.3, 6.5, 3.5, 0.4, 0.3, -1, -1, 1.5, 1.2, 1.3, -1, 0.3, 0.7, 0.5, 0.4, 0.4, -1, 0.5, 0.8, 0.4, 1.0, 0.3, -1, 1.1, 0.4, 1.7, 0.5, 1.8, -1, 1.0, 0.7, 0.4, 0.6, 0.2, -1, 0.8, 0.7, 0.8, 0.9, 0.5, -1, 0.8, 1.4, 0.7, -1, 1.3, 1.5, 0.4, 0.7, -1, 1.5, 0.5, 0.5, 0.9, -1, -1, -1, -1, 0.9, -1, 4.2, -1, -1, 0.7, 0.7, -1, 0.6, 0.4, 0.2, -1, 0.7, 0.6, 1.2, 1.0, -1, 1.8, -1, 0.7, 0.4, 0.4, -1, 0.4, -1, 0.6, -1, -1, 0.6, 0.1, 0.7, 0.4, 0.9, -1, 0.4, -1, 0.6, 0.5, 1.2, 0.5, 1.0, 0.6, 0.5, 0.5, -1, 2.5, 1.1, 0.4, -1, 0.5, 0.6, 0.4, 0.4, -1, 0.7, 1.0, 0.3, -1, -1, 1.2, -1, -1, -1, 0.4, 0.6, -1, 2.0, -1, 0.9, -1, -1, 1.7, 1.1, 0.4, 0.5, 0.8, 0.8, 1.6, -1, -1, 0.2, 1.5, -1, -1, 1.2, 0.4, 0.5, 0.3, 1.3, 0.8, 0.8, -1, -1, 0.6, -1, 0.5, 0.5, 0.4, -1, -1, 0.1, 2.2, -1, 2.5, 0.3, 0.4, -1, -1, 0.6, 0.8, 0.3, 0.8, -1, 0.8, -1, 0.4, 0.4, -1, 0.5, 0.5, -1, -1, -1, 2.5, 1.5, -1, -1, 1.5, 0.5, 2.0, -1

In [28]:
# Check if 2 arrays are the same output, if so return one of them if it is not empty, bias towards A
def majorityVoteCheck(index):
    A_val = A_unfiltered_array[index]
    PE_val = PE_unfiltered_array[index]
    AP_val = AP_unfiltered_array[index]
    
    if A_val == PE_val or A_val == AP_unfiltered_array[index]: # If A or either match
        if A_val: return A_val
        elif PE_val: return PE_val
        elif AP_val: return AP_val
    elif PE_val == AP_val: # If A&P and A match
        if PE_val: return PE_val
        elif AP_val: return AP_val
    else: # None match, return A
        return A_val
            
# Only call other 2 columns to check for false positives (0=PE 1=A&P 2=A)
def prioritizeColumn(index, prioritize_column_index):
    col_arrays = [PE_unfiltered_array, AP_unfiltered_array, A_unfiltered_array]
    col_prioritize = col_arrays[prioritize_column_index]
    
    if "y" in col_prioritize[index]:
        # This is where other 2 are called by GPT in other file
        other_indices = [i for i in [0, 1, 2] if i != prioritize_column_index]
        for i in other_indices:
            if "y" in col_arrays[i][index]:
                return "y"
        return "n"  # Prioritized Column said Yes, but others disagreed, possible false positive
    else:
        return "n"
    
import re

# Convert mm to cm if needed and round to 2 decimal places.
def convert_size(val, unit):
    if unit:
        unit = unit.lower()
        if unit in ['mm', 'millimeters']:
            val /= 10
    return round(val, 2)

# Extract  sizes, covers multiple input types
def extract_numeric_sizes(size_string):
    size_matches = re.findall(
        r'(\d*\.?\d+)\s*to\s*(\d*\.?\d+)?\s*(cm|mm|millimeters)?'                             # e.g. "8 to 10 mm"
        r'|(\d*\.?\d+)\s*(cm|mm|millimeters)?\s*[x×]?\s*(\d*\.?\d+)?\s*(cm|mm|millimeters)?'  # e.g. "4 x 5 mm"
        r'|(\d*\.?\d+)\s*-\s*(\d*\.?\d+)\s*(cm|mm|millimeters)?',                             # e.g. "2-3 mm"
        size_string,
        re.IGNORECASE
    )

    numeric_sizes = []

    for match in size_matches:
        # Case 1: "X to Y unit"
        if match[0]:
            val1 = convert_size(float(match[0]), match[2])
            val2 = convert_size(float(match[1]), match[2]) if match[1] else val1
            numeric_sizes.append(max(val1, val2))

        # Case 2: "X x Y unit"
        elif match[3]:
            val1 = convert_size(float(match[3]), match[4])
            val2 = convert_size(float(match[5]), match[6]) if match[5] else val1
            numeric_sizes.append(max(val1, val2))

        # Case 3: "X - Y unit"
        elif match[7]:
            val1 = convert_size(float(match[7]), match[9])
            val2 = convert_size(float(match[8]), match[9])
            numeric_sizes.append(max(val1, val2))

    return numeric_sizes

# Processes GPT responses and returns a final largest size in cm
def evaluate_size_responses(output, true_results, sample_size, skip_patients, skip_patients_array):
    output_final = [-2 for i in range(len(output))] # Default to -2 (invalid GPT response)
    mmArr = [0 for i in range(len(output))]

    for i in range(min(sample_size, len(output))):
        if skip_patients and i in skip_patients_array:
            continue

        response = output[i]
        if not isinstance(response, str) or response.strip().lower() in ["na", "n/a", ""]:
            continue

        numeric_sizes = extract_numeric_sizes(response)

        if not numeric_sizes:
            output_final[i] = -2  # Invalid GPT response
            continue

        max_size = max(numeric_sizes)
        output_final[i] = round(max_size, 2)

    return output_final

only_PE = [0 for i in range(sample_size)]
only_AP = [0 for i in range(sample_size)]
only_A = [0 for i in range(sample_size)]
majority_vote = [0 for i in range(sample_size)]
prioritize_PE = [0 for i in range(sample_size)]
prioritize_AP = [0 for i in range(sample_size)]
prioritize_A = [0 for i in range(sample_size)]

for i in range(0, sample_size): # This is where all the types of votes are run
    #only_PE[i] = PE_unfiltered_array[i]
    #only_AP[i] = AP_unfiltered_array[i]
    only_A[i] = A_unfiltered_array[i]
    if len(PE_unfiltered_array) > 0 and len(AP_unfiltered_array) > 0 and len(A_unfiltered_array) > 0:
        majority_vote[i] = majorityVoteCheck(i)
        prioritize_PE[i] = prioritizeColumn(i, 0)
        prioritize_AP[i] = prioritizeColumn(i, 1)
        prioritize_A[i] = prioritizeColumn(i, 2)
    else:
        only_A_flag = True

# For size question only, change to be called better later
output_final_PE = evaluate_size_responses(PE_unfiltered_array, true_results, sample_size, skip_patients, skip_patients_array)
output_final_AP = evaluate_size_responses(AP_unfiltered_array, true_results, sample_size, skip_patients, skip_patients_array)
output_final_A = evaluate_size_responses(A_unfiltered_array, true_results, sample_size, skip_patients, skip_patients_array)
        
print("Done")
#print(majority_vote)
#print(prioritize_column)
print(output_final_PE)
print(output_final_AP)
print(output_final_A)
# prioritize_column -> prioritize_PE, prioritize_AP, prioritize_A

Done
[0.8, 0.6, 0.3, 6.5, 1.6, 0.4, 0.3, -2, -2, 1.5, 1.2, -2, -2, 0.3, 0.7, 0.5, 0.4, 0.4, 1.0, 0.5, 0.8, 0.4, -2, -2, 1.5, 1.1, 0.4, 1.7, 0.5, 1.8, -2, 1.0, -2, -2, -2, 0.2, -2, 0.8, -2, -2, 0.9, 0.5, -2, 0.8, 1.4, -2, -2, 1.3, 1.5, -2, -2, 2.0, -2, 0.5, 0.5, -2, -2, -2, -2, -2, 0.9, -2, -2, -2, 1.0, 0.7, -2, -2, 0.6, 0.4, 0.2, -2, 0.7, -2, 1.2, 1.0, -2, 1.8, -2, 0.7, -2, 0.4, -2, 0.4, -2, 0.6, -2, -2, -2, -2, 0.7, 0.4, 0.9, 2.0, 0.4, 0.5, 0.6, -2, 1.2, 0.5, 5.0, -2, -2, 0.5, -2, 2.5, 1.1, -2, -2, 0.5, -2, 0.4, 0.4, 1.5, 0.7, 1.0, 0.3, 1.0, -2, 1.2, -2, -2, -2, -2, 0.6, 1.0, 0.9, -2, 0.9, -2, -2, -2, -2, -2, 0.5, -2, -2, -2, -2, -2, -2, 1.5, -2, 3.0, 1.2, 0.4, 0.5, 0.3, 1.3, 0.8, -2, -2, -2, 0.6, -2, -2, 0.5, -2, 1.0, 0.6, 0.1, 2.2, 3.0, 2.5, -2, 0.4, -2, -2, 0.6, 0.8, -2, 0.8, -2, 0.8, -2, 0.4, 0.4, -2, 0.5, 0.5, -2, -2, -2, 2.5, -2, -2, -2, -2, 0.5, 2.0, -2, -2, 0.6, -2, 0.5, 0.6, 1.0, 1.0, -2, 0.6, -2, -2, -2, 0.5, -2, 0.6, -2, 5.0, 1.1, 1.7, -2, -2, 0.5, -2, -2, 0.4, -2, 1.2, 0.5

In [29]:
def confusionMatrix(vote_array):
    TP = TN = FP = FN = 0
    for i in range(0, min((sample_size), len(Cat))):
        if vote_array[i] == true_results[i]: # True+/True-
            if vote_array[i] == "y": # True Positive
                TP += 1
            elif vote_array[i] == "n": # True Negative
                TN += 1
        elif(vote_array[i] != true_results[i] and true_results[i] != "NaN error" and vote_array[i] != "error"): # False+/False-
            if vote_array[i] == "y": # False Positive
                FP += 1
            elif vote_array[i] == "n": # False Negative
                FN += 1
    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN}

def confusionMetrics(confusion_matrix):
    TP = confusion_matrix["TP"]
    TN = confusion_matrix["TN"]
    FP = confusion_matrix["FP"]
    FN = confusion_matrix["FN"]
    
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / (TP + TN + FP + FN) if total else 0 # How often did model predict correctly
    precision = TP / (TP + FP) if (TP + FP) else 0 # When model predicted Y, how often was it correct
    recall = TP / (TP + FN) if (TP + FN) else 0 # How many of the true Y cases did model correctly identify
    F1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0 # Balance between Precision and Recall
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "F1": F1}

def formatConfusionMatrix(matrix, metrics, label):     
    output = [f"\nConfusion Matrix {label}:"]
    for name, value in matrix.items():
        output.append(f"{name}: {value}")
    output.append(f"\nConfusion Matrix Metrics {label}:")
    for name, value in metrics.items():
        output.append(f"{name}: {(value * 100):.2f}")
    return "\n".join(output)

def evaluate_size_accuracy(predictions, true_results):
    correct = 0
    total = len(predictions)
    
    for i in range(total):
        if predictions[i] == true_results[i]:  # If the predicted size matches the true result
            correct += 1
    
    accuracy = (correct / total) * 100 if total else 0  # Percentage of correct predictions
    return accuracy

def generateMatrices():
    with open("matrices/ae_matrix(3).txt", "a") as log_file:
        if only_A_flag == False:
            confusion_matrix_PE = confusionMatrix(only_PE)
            confusion_metrics_PE = confusionMetrics(confusion_matrix_PE)
            output_PE = formatConfusionMatrix(confusion_matrix_PE, confusion_metrics_PE, "PE")
            print(output_PE)
            log_file.write(output_PE + "\n")

            confusion_matrix_AP = confusionMatrix(only_AP)
            confusion_metrics_AP = confusionMetrics(confusion_matrix_AP)
            output_AP= formatConfusionMatrix(confusion_matrix_AP, confusion_metrics_AP, "A&P")
            print(output_AP)
            log_file.write(output_AP + "\n")

        confusion_matrix_A = confusionMatrix(only_A)
        confusion_metrics_A = confusionMetrics(confusion_matrix_A)
        output_A = formatConfusionMatrix(confusion_matrix_A, confusion_metrics_A, "Addendum")
        print(output_A)
        log_file.write(output_A + "\n")

        if only_A_flag == False:
            confusion_matrix_majority = confusionMatrix(majority_vote)
            confusion_metrics_majority = confusionMetrics(confusion_matrix_majority)
            output_majority = formatConfusionMatrix(confusion_matrix_majority, confusion_metrics_majority, "Majority")
            print(output_majority)
            log_file.write(output_majority + "\n")

            prioritize_outputs = {
                "Prioritize PE": prioritize_PE,
                "Prioritize AP": prioritize_AP,
                "Prioritize A": prioritize_A
            }

            for name, results in prioritize_outputs.items():
                cm = confusionMatrix(results)
                metrics = confusionMetrics(cm)
                output = formatConfusionMatrix(cm, metrics, name)
                print(output)
                log_file.write(output + "\n")
                
#generateMatrices()
accuracy = evaluate_size_accuracy(output_final_PE, true_results)
print(f"Accuracy: {accuracy:.2f}%")
# Accuracy: 48.98% no skips
# Accuracy: 47.96% with skips, clearly I implemented something in here wrong lol, either regex or comparisons were done wrong
# Just test in open_ai_nicky.ipynb for Size cause there's no time to fix this
# Running in open_ai_nicky.ipynb gave Accuracy = 98.18% so yeah something was done incorrectly here

Accuracy: 47.96%
